# F1 Fantasy Program

## ReadMe

The goal of this program is to webscrape, and gather, F1 data. The data would then be saved, transformed, and analysed.

# Imports

#### General

In [16]:
import sys
sys.path.append('modules')

from datetime import datetime
import requests

#### FastF1

In [2]:
import fastf1_extract as ff1
import glob
import pandas as pd

#### Webscrape

In [3]:
import f1_webscrape_extract as f1ws

#### MySQL

In [4]:
import f1stats_database as f1db

### Ignore Warnings

In [5]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Global Variables & Settings

### Loaded races logfile name

In [6]:
loaded_races_logfile = 'loaded_races_logfile.txt'

# Save to CSV file

In [7]:
def save_to_csv(target_csv, data_to_load):
    data_to_load.to_csv(target_csv) #, mode='a', header=False, index=False)

# Extract: FastF1

### Log Loaded Race

In [8]:
def log_loaded_race(race_number, loaded_races_logfile = 'loaded_races_logfile.txt'):
	timestamp_format = '%Y-%h-%d-%H:%M:%S' #Year-Monthname-Day-Hour-Minute-Second
	now = datetime.now()
	timestamp = now.strftime(timestamp_format)
	with open(loaded_races_logfile, 'a') as log:
		log.write(timestamp + ','+str(race_number)+'\n')

### Read logfile

In [9]:
def read_loaded_races_logfile(loaded_races_logfile = 'loaded_races_logfile.txt'):
	loaded_races = []
	with open(loaded_races_logfile, 'r') as log:
		for line in log.readlines():
			# print(line)
			fields = line.strip().split(',')
			loaded_races.append(int(fields[1]))
	
	return loaded_races

### Find missing races

In [10]:
def find_missing_races(races_list: list, season: int):
	# given a list of races, and the count of completed races, it finds the numbers that are missing
	if len(races_list) == 0:
		races_list = [0]
	missing = set(range(min(races_list), ff1.get_completed_events_count(season)+1)) - set(races_list)
	return list(missing)

# Webscraping

In [11]:
# f1ws.TODO_SORT_INTO_FUNCTIONS()

# Transform: Extract select columns from Data, into CSV files

In [12]:
def extract_ff1_data_into_csv(season=2023, session_type='R'):
	event_number = 0

	loaded_races = read_loaded_races_logfile()
	# print(loaded_races)
	missing_races = find_missing_races(loaded_races, season)
	# print(missing_races)

	races_csv_folder = 'races_csv/'
	for event_num in missing_races:
		session = ff1.get_session_data(season, event_num, session_type) # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! TODO only relevant columns must be saved into the csv files
		save_to_csv(races_csv_folder+str(session.event.gp)+'_'+str(session.event.date)+'.csv',session.results)
		log_loaded_race(event_num) # maybe move this to after the data is actually loaded into the database

# MySQL Database f1stats

In [13]:
# connect to MySQL database and return db_object
mydb = f1db.connect_to_database()



# Execute Functions
def execute_db_insert_functions():
    with mydb.cursor() as mycursor:
        try:
            # Call insert functions here
            mydb.commit()
        except Exception as e:
            print(f"An error occurred: {e}")
            mydb.rollback()
            raise



def get_standings():
    # Define the API endpoint URL
    url = "https://fantasy.formula1.com/api/f1/2021/standings"

    try:
        # Make a GET request to the API endpoint
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON data from the response
            data = response.json()

            # Access the data as needed
            standings = data['standings']
            return standings
        else:
            # Handle the error if the request was not successful
            print(f"Failed to retrieve data from the API. Response code: {response.status_code}")
            return None
    except Exception as e:
        # Handle any unexpected exceptions
        print(f"An error occurred while connecting to the API: {e}")
        return None

In [ ]:
def insert_data_from_dataframe(cursor, dataframe, table_name):
    for index, row in dataframe.iterrows():
        data_row = row.to_dict()
        if table_name == 'teams':
            f1db.insert_into_teams(cursor, data_row)
        elif table_name == 'team_price_history':
            f1db.insert_into_team_price_history(cursor, data_row, team_id_fk=row['team_id_fk'])
        elif table_name == 'drivers':
            f1db.insert_into_drivers(cursor, data_row, team_id_fk=row['team_id_fk'])
        elif table_name == 'driver_price_history':
            f1db.insert_into_driver_price_history(cursor, data_row, driver_id_fk=row['driver_id_fk'])
        elif table_name == 'circuit':
            f1db.insert_into_circuit(cursor, data_row)
        elif table_name == 'race':
            f1db.insert_into_race(cursor, data_row, circuit_id_fk=row['circuit_id_fk'])
        elif table_name == 'race_results':
            f1db.insert_into_race_results(cursor, data_row, race_id_fk=row['race_id_fk'], driver_id_fk=row['driver_id_fk'], team_id_fk=row['team_id_fk'])
        elif table_name == 'weather':
            f1db.insert_into_weather(cursor, data_row, race_id_fk=row['race_id_fk'])
        else:
            raise ValueError(f"Invalid table name: {table_name}")

In [16]:
import uuid
def generate_uuid():
    return uuid.uuid4().bytes # this will return a random binary number

In [17]:
print(generate_uuid())

b'\xc8\xd6\x0fR5\x14@4\x89\xb5~L\xe2[F\x03'


## Logging

In [14]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' #Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open('ETL_log.txt', 'a') as log: 
        log.write(timestamp + ','+str(message)+'\n')

# ETL Pipeline

In [15]:
import os
import pandas as pd
def etl():
    ff1.enable_cache()

    # Extract Data
    # log('Start Extract Step')
    extract_ff1_data_into_csv() # double check this later
    race_csv_files = glob.glob('/races_csv/*.csv')
    list_of_race_dfs = []

    # Load the list of already processed files
    if os.path.exists('processed_files.txt'):
        with open('processed_files.txt', 'r') as f:
            processed_files = f.read().splitlines()
    else:
        processed_files = []

    for file in race_csv_files:
        if file not in processed_files:
            df = pd.read_csv(file)
            list_of_race_dfs.append(df) # add race to list of race dfs
            # Add the file to the list of processed files
            processed_files.append(file)

    # Save the updated list of processed files
    with open('processed_files.txt', 'w') as f:
        f.write('\n'.join(processed_files))


    # insert webscraping
    # log('End Extract Step')



    # Transform
    # log('Start Transfrom Step')

    # Create dataframes for each table
    
    teams_df = [['team_name']]
    # teams_df = 
    # team_price_history_df = 
    # drivers_df = 
    # driver_price_history_df = 
    # circuit_df = 
    # race_df = 
    # race_results_df = 
    # weather_df = 

    # log('End Transform Step')


    # Load
    # log('Start Load Step')
    # Connect to the database
    mydb = f1db.connect_to_database()
    cursor = mydb.cursor()


    # Insert data into the tables
    insert_data_from_dataframe(cursor, teams_df, 'teams')
    insert_data_from_dataframe(cursor, team_price_history_df, 'team_price_history')
    insert_data_from_dataframe(cursor, drivers_df, 'drivers')
    insert_data_from_dataframe(cursor, driver_price_history_df, 'driver_price_history')
    insert_data_from_dataframe(cursor, circuit_df, 'circuit')
    insert_data_from_dataframe(cursor, race_df, 'race')
    insert_data_from_dataframe(cursor, race_results_df, 'race_results')
    insert_data_from_dataframe(cursor, weather_df, 'weather')

    # Commit changes and close the connection
    mydb.commit()
    cursor.close()
    mydb.close()
    
    # log('End Load Step')